In [ ]:
# !pip install -r https://raw.githubusercontent.com/anujavenkatachalam04/chvi_vbd_rj/main/requirements.txt

In [ ]:
import os
import pandas as pd
import geopandas as gpd
import requests
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive')
import geopandas as gpd
from shapely.geometry import Point
import uuid
from dotenv import load_dotenv
import openmeteo_requests
import requests_cache
from retry_requests import retry
from tqdm import tqdm


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
os.chdir("/content/drive/MyDrive/CHVI")


In [ ]:
blocks_shp=gpd.read_file("5_Shapefiles/SUBDISTRICT_11/Rajasthan_Blocks.geojson")

In [12]:
grid=gpd.read_file("5_Shapefiles/Rajasthan_10kmx10km_grid.geojson")

In [16]:
# Load API key from .env  only works for gh
# load_dotenv()
# OPENMETEO_API_KEY = os.getenv("OPENMETEO_API_KEY")

In [17]:
grid

,NAME,DISTRICT,C_CODE11,geometry
0,Jaisalmer,Jaisalmer,0811400574000000,POINT (69.58509 26.75094)
1,Jaisalmer,Jaisalmer,0811400574000000,POINT (69.5808 26.84083)
2,Jaisalmer,Jaisalmer,0811400574000000,POINT (69.5765 26.93071)
3,Jaisalmer,Jaisalmer,0811400574000000,POINT (69.57218 27.02059)
4,Jaisalmer,Jaisalmer,0811400574000000,POINT (69.56784 27.11047)
...,...,...,...,...
3415,Rajakhera,Dhaulpur,0810600519000000,POINT (78.02682 26.73203)
3416,Rajakhera,Dhaulpur,0810600519000000,POINT (78.02922 26.82219)
3417,Rajakhera,Dhaulpur,0810600519000000,POINT (78.1297 26.82)
3418,Rajakhera,Dhaulpur,0810600519000000,POINT (78.13219 26.91015)


# Extract ERA5 data at 10km x 10km resolution

In [21]:
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry

# Setup Open-Meteo API client
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)


def fetch_era5_data(lat, lon, start_date, end_date, name, district, code, api_key):
    """
    Fetch daily ERA5 data for a single coordinate (lat/lon)
    Returns a Pandas DataFrame with all variables and metadata.
    """

    url = "https://customer-archive-api.open-meteo.com/v1/archive"

    # Define the list of relevant daily climate variables (for vector-borne disease modeling)
    daily_vars = [
        "temperature_2m_mean", "temperature_2m_max", "temperature_2m_min",
        "rain_sum", "precipitation_sum", "relative_humidity_2m_mean",
        "apparent_temperature_mean", "precipitation_hours",
        "apparent_temperature_max", "apparent_temperature_min",
        "et0_fao_evapotranspiration", "wind_speed_10m_max",
        "wind_gusts_10m_max", "wind_direction_10m_dominant",
        "shortwave_radiation_sum", "sunshine_duration"
    ]

    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date,
        "end_date": end_date,
        "daily": daily_vars,
        "models": "era5",
        "apikey": api_key
    }

    try:
        # Call API
        responses = openmeteo.weather_api(url, params=params)
        response = responses[0]
        daily = response.Daily()

        # Build time range
        date_index = pd.date_range(
            start=pd.to_datetime(daily.Time(), unit="s", utc=True),
            end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=daily.Interval()),
            inclusive="left"
        )

        # Map variables dynamically based on list order
        daily_data = {"date": date_index}
        for i, var in enumerate(daily_vars):
            daily_data[var] = daily.Variables(i).ValuesAsNumpy()

        # Add metadata
        daily_data.update({
            "NAME": name,
            "DISTRICT": district,
            "C_CODE11": code,
            "latitude": lat,
            "longitude": lon
        })

        return pd.DataFrame(daily_data)

    except Exception as e:
        print(f"Error fetching data for {name} ({district}): {e}")
        return pd.DataFrame()


In [22]:
start_date = "2024-01-01"
end_date = "2025-10-01"
api_key=""


all_results = []
for idx, row in tqdm(grid.iterrows(), total=len(grid)):
    lat = row.geometry.y
    lon = row.geometry.x
    df = fetch_era5_data(lat, lon, start_date, end_date, row.NAME, row.DISTRICT, row.C_CODE11, api_key)
    if not df.empty:
        all_results.append(df)

# Combine
era5_grid_df = pd.concat(all_results, ignore_index=True)

100%|██████████| 3420/3420 [20:06<00:00,  2.83it/s]


In [ ]:
def get_season(month):
    if month in [3, 4, 5]:
        return 'Summer (Mar-May)'
    elif month in [6, 7, 8]:
        return 'Monsoon (Jun-Aug)'
    elif month in [9, 10, 11]:
        return 'Post-Monsoon (Sep-Nov)'
    else:
        return 'Winter (Dec-Feb)'

In [ ]:
era5_grid_df["date"]=pd.to_datetime(era5_grid_df["date"])

In [ ]:
era5_grid_df["season"]=era5_grid_df["date"].dt.month.apply(get_season)

In [ ]:
era5_grid_df["diurnal_temperature_range_computed"]=era5_grid_df["temperature_2m_max"]-era5_grid_df["temperature_2m_min"]

In [36]:
era5_grid_df.to_csv("1_Data/Climate/Raw/openmeteo_era5_grid_data_10kmx10km_2024-01-01_to_2025-10-01.csv", index=False)

# Process climate data

a. By block & day

In [37]:
era5_grid_df.columns

Index(['date', 'temperature_2m_mean', 'temperature_2m_max',
       'temperature_2m_min', 'rain_sum', 'precipitation_sum',
       'relative_humidity_2m_mean', 'apparent_temperature_mean',
       'precipitation_hours', 'apparent_temperature_max',
       'apparent_temperature_min', 'et0_fao_evapotranspiration',
       'wind_speed_10m_max', 'wind_gusts_10m_max',
       'wind_direction_10m_dominant', 'shortwave_radiation_sum',
       'sunshine_duration', 'NAME', 'DISTRICT', 'C_CODE11', 'latitude',
       'longitude', 'season', 'diurnal_temperature_range_computed'],
      dtype='object')

In [39]:
# Variables to aggregate (excluding metadata columns)
value_cols = [
    'temperature_2m_mean', 'temperature_2m_max', 'temperature_2m_min',
    'rain_sum', 'precipitation_sum', 'relative_humidity_2m_mean',
    'apparent_temperature_mean', 'precipitation_hours',
    'apparent_temperature_max', 'apparent_temperature_min',
    'et0_fao_evapotranspiration', 'wind_speed_10m_max',
    'wind_gusts_10m_max', 'wind_direction_10m_dominant',
    'shortwave_radiation_sum', 'sunshine_duration',
    'diurnal_temperature_range_computed'
]

# Group and compute mean
agg_df = (
    era5_grid_df
    .groupby(['DISTRICT', 'NAME', 'C_CODE11', 'date'], as_index=False)[value_cols]
    .mean()
)

# Add "mean_" prefix to all aggregated variable columns
agg_df = agg_df.rename(columns={col: f"mean_{col}" for col in value_cols})

agg_df.head()


,DISTRICT,NAME,C_CODE11,date,mean_temperature_2m_mean,mean_temperature_2m_max,mean_temperature_2m_min,mean_rain_sum,mean_precipitation_sum,mean_relative_humidity_2m_mean,...,mean_precipitation_hours,mean_apparent_temperature_max,mean_apparent_temperature_min,mean_et0_fao_evapotranspiration,mean_wind_speed_10m_max,mean_wind_gusts_10m_max,mean_wind_direction_10m_dominant,mean_shortwave_radiation_sum,mean_sunshine_duration,mean_diurnal_temperature_range_computed
0,Ajmer,Ajmer,0811900607000000,2024-01-01 00:00:00+00:00,13.758790,20.770250,8.845250,0.0,0.0,72.577873,...,0.0,19.951094,7.155656,2.564523,9.624511,22.139999,69.226219,15.500000,34828.792969,11.924999
1,Ajmer,Ajmer,0811900607000000,2024-01-02 00:00:00+00:00,12.113999,17.320250,8.145249,0.0,0.0,79.751427,...,0.0,16.239964,6.576055,2.220969,10.990425,22.860001,97.166710,14.924999,34635.679688,9.174999
2,Ajmer,Ajmer,0811900607000000,2024-01-03 00:00:00+00:00,11.961917,18.245249,7.070250,0.0,0.0,77.487595,...,0.0,17.777025,5.165407,2.199089,8.274967,22.319998,68.480377,14.139999,34884.175781,11.174999
3,Ajmer,Ajmer,0811900607000000,2024-01-04 00:00:00+00:00,11.155666,17.095251,6.520250,0.0,0.0,75.663658,...,0.0,16.232700,4.369008,2.353657,8.422462,19.080000,64.216011,15.500000,34915.179688,10.575000
4,Ajmer,Ajmer,0811900607000000,2024-01-05 00:00:00+00:00,11.165042,16.895250,6.495250,0.0,0.0,75.972206,...,0.0,15.987430,4.220113,2.377067,10.158813,23.760000,62.649609,15.950001,34948.300781,10.400000


In [40]:
agg_df["season"]=agg_df["date"].dt.month.apply(get_season)

In [44]:
agg_df.to_csv("1_Data/Climate/Clean/daily_climate_block.csv", index=False)

# Climate - Aggregate API data (at grid-level) to district & day

b. By district & day

In [45]:
# Variables to aggregate (excluding metadata columns)
value_cols = [
    'temperature_2m_mean', 'temperature_2m_max', 'temperature_2m_min',
    'rain_sum', 'precipitation_sum', 'relative_humidity_2m_mean',
    'apparent_temperature_mean', 'precipitation_hours',
    'apparent_temperature_max', 'apparent_temperature_min',
    'et0_fao_evapotranspiration', 'wind_speed_10m_max',
    'wind_gusts_10m_max', 'wind_direction_10m_dominant',
    'shortwave_radiation_sum', 'sunshine_duration',
    'diurnal_temperature_range_computed'
]

# Group and compute mean
dist_agg_df = (
    era5_grid_df
    .groupby(['DISTRICT', 'date'], as_index=False)[value_cols]
    .mean()
)

# Add "mean_" prefix to all aggregated variable columns
dist_agg_df = dist_agg_df.rename(columns={col: f"mean_{col}" for col in value_cols})

dist_agg_df.head()


,DISTRICT,date,mean_temperature_2m_mean,mean_temperature_2m_max,mean_temperature_2m_min,mean_rain_sum,mean_precipitation_sum,mean_relative_humidity_2m_mean,mean_apparent_temperature_mean,mean_precipitation_hours,mean_apparent_temperature_max,mean_apparent_temperature_min,mean_et0_fao_evapotranspiration,mean_wind_speed_10m_max,mean_wind_gusts_10m_max,mean_wind_direction_10m_dominant,mean_shortwave_radiation_sum,mean_sunshine_duration,mean_diurnal_temperature_range_computed
0,Ajmer,2024-01-01 00:00:00+00:00,13.936296,20.796059,9.379198,0.0,0.0,79.207817,13.051352,0.0,20.687984,7.984906,2.374903,9.127807,21.298603,68.094322,14.828721,34783.625000,11.416861
1,Ajmer,2024-01-02 00:00:00+00:00,12.300927,17.301291,8.604198,0.0,0.0,82.898071,11.145679,0.0,16.509884,7.128253,2.095210,9.981946,22.332558,88.350090,14.198256,32940.289062,8.697093
2,Ajmer,2024-01-03 00:00:00+00:00,12.189929,18.262337,7.572221,0.0,0.0,79.695168,10.879541,0.0,17.971647,5.828930,2.021272,8.650051,22.102325,64.787643,13.042791,34463.039062,10.690117
3,Ajmer,2024-01-04 00:00:00+00:00,11.350103,17.072222,6.604198,0.0,0.0,77.507935,9.751969,0.0,16.376350,4.555708,2.318121,9.226160,19.456743,64.911667,15.373488,34974.789062,10.468023
4,Ajmer,2024-01-05 00:00:00+00:00,11.184746,16.890825,6.618732,0.0,0.0,79.686485,9.636365,0.0,16.235310,4.538643,2.224337,9.392123,23.211628,58.796307,15.251628,35013.699219,10.272094


In [46]:
dist_agg_df["season"]=dist_agg_df["date"].dt.month.apply(get_season)

In [47]:
dist_agg_df.to_csv("1_Data/Climate/Clean/daily_climate_district.csv", index=False)

In [ ]:
# The End!